# Import Libraries

In [ ]:
# import os
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import time
import pylab as pl
import pandas as pd
from numpy import loadtxt, zeros, ones, array, linspace, logspace
from pylab import scatter, show, title, xlabel, ylabel, plot, contour
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn import cross_validation, metrics
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Tweeter Data Streaming

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


consumer_key = ' '
consumer_secret = ''
access_token = ''
access_token_secret = ''


class twitterListener(StreamListener):

    def on_data(self, data):
        print (data)
        return True

    def on_error(self, status):
        print (status)


l = twitterListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)

stream.filter(languages=["en"], track=['medicine', 'meds', 'medical', 'infected', 'health', 'pill', 'pills', 'drug', 'drugs',\
                                        'surgeon', 'surgeons', 'disease', 'diseases', 'virus', 'viruses', 'bacteria',\
                                        'vaccine', 'vaccines', 'doctor', 'doctors', 'patient', 'patients','hospital'\
                                        'clinic', 'prescription', 'sick', 'illness', 'allergy', 'fever', 'flu', 'autism'\
                                        'cancer', 'diabetes', 'HIV', 'injection', 'antibiotic', 'disorder'])

# Save Tweets Information into CSV and Extract from json format

In [ ]:
import json
import pandas as pd


def readFromFile(filePath):
    # Read from the file and append the data to a list

    tweets_data_path = filePath
    tweets_data = []
    tweets_file = open(tweets_data_path, "r")

    for line in tweets_file:
        # append json file data to a list
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue

    return tweets_data
    

def listToDataFrame(lst, headerList):
    # Read from a list and create a Pandas dataframe with the corresponding headers 
    return pd.DataFrame(lst, columns=header)


header = ['text', 'created_at', 'user']
filePath = r'C:\Users\twittermed.txt'

#print(readFromFile(filePath)[:5])
dataList = readFromFile(filePath)
tweets = listToDataFrame(dataList, header)
tweets['is_retweeted'] = [True if 'retweeted_status' in d else False for d in dataList]
#tweets['bio'] = [d.get('description', False) for d in list(tweets.user)]
#[True if 9 in i else False for i in a]
#print(tweets.head())
print(len(tweets))

tweets = tweets.dropna(axis=0, how='any') #delete NAN rows
print(len(tweets))

tweets['screen_name'] = [d.get('screen_name') for d in list(tweets.user)]
tweets['followers_count'] = [d.get('followers_count') for d in list(tweets.user)]
tweets['friends_count'] = [d.get('friends_count') for d in list(tweets.user)]
tweets['statuses_count'] = [d.get('statuses_count') for d in list(tweets.user)]
tweets['favourites_count'] = [d.get('favourites_count') for d in list(tweets.user)]
tweets['user_created_at'] = [d.get('created_at') for d in list(tweets.user)]
tweets['bio'] = [d.get('description') for d in list(tweets.user)]
tweets = tweets[tweets.is_retweeted == False]


#tweets['verified'] = [d.get('verified') for d in list(tweets.user)]
#tweets = tweets.dropna(axis=0, how='any') #delete NAN rows
print(tweets.shape)

tweets.to_csv("twittertest2.csv", encoding='utf8')

# Read CSV file (extracted information)

In [ ]:
#import json
import pandas as pd
#import numpy as np
from datetime import datetime


tweets = pd.read_csv("twittertest2.csv", encoding = "UTF-8", engine='python')
############# remove unwanted tweets
tweets.head()
tweets.shape

# Remove tweets which has Null Bio values 

In [ ]:
tweets = tweets.dropna(axis=0, how='any')
tweets.shape

# Processing Twitter User Bio or Description

In [ ]:
import re
def addd(x): 
    #print(x['bio'])
    x['bio'] = x['bio'].lower()
    return re.sub ('[^a-zA-Z]', ' ', x['bio'])

tweets['bio'] = tweets.apply(addd, 1)
tweets = tweets.dropna(axis=0, how='any')
print(tweets.shape)


# Tweet Data Preprosseing

In [ ]:
#import json
import pandas as pd
#import numpy as np
from datetime import datetime


#tweets = pd.read_csv("twittertest2.csv",  encoding = "ISO-8859-1", header = 0, engine='python')


############# remove unwanted tweets

#### Remove RTs:
print(len(tweets))

#print(len(tweets))
#tweets = tweets[tweets.is_retweeted == False]
print(len(tweets))

#### Check the membership time

def __datetime(date_str):
    return datetime.strptime(date_str, '%a %b %d %H:%M:%S +0000 %Y')

def apply_on_df(x):
    return (__datetime(x['created_at']) - __datetime(x['user_created_at'])).days
    

tweets['active_days'] = tweets.apply(apply_on_df, axis = 1)
#print(tweets.head())

#print(len(tweets))
tweets = tweets[tweets.active_days > 90 ]
#print(len(tweets))


#### Check friends and following

#print(len(tweets))
tweets = tweets[tweets.followers_count - tweets.friends_count < 700]
#print(len(tweets))

##### Remove bots

#print(len(tweets))
tweets = tweets[tweets.statuses_count // tweets.active_days < 15]
print(len(tweets))
print(tweets.shape)


###### Check for wanted tweets

#keywordList = ['medical', 'physician' , 'doctor', 'nurse']
#tweets['possible'] = tweets[True if any(k in tweets.bio) else False for k in keywordList]

# Filter User who has valid Bio information 

In [ ]:
tweets = tweets[tweets['bio'].str.contains("physician|hospital|doctor|clinic|nurse|specialist|gynecologist|dentist|medicine|surgeon")]
print(tweets.shape)

In [ ]:
tweet2 = tweets

# Extract Previous tweets of Twitter User (Assumed Expert)

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json


consumer_key = ''
consumer_secret = ''
access_token = ''

access_token_secret = ''



auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)




def process_or_store(tweet):
    print(json.dumps(tweet))
    
    
api = tweepy.API(auth)

def checkStatus(x):
    try:
        status = []
        dictioanry = {}
        stuff = api.user_timeline(x['screen_name'], count = 10, include_rts = False)
        for s in stuff:
            if s.str.contains("medicine|meds|medical|infected|health|pill|pills|drug|drugs|\
                            surgeon|surgeons|disease|diseases|virus|viruses|bacteria|vaccine|\
                            vaccines|doctor|doctors|patient|patients|hospital|clinic|prescription|\
                            sick|illness|allergy|fever|flu|autism|cancer|diabetes|HIV|injection|\
                            antibiotic|disorder|physician|dentist|gynecologist|surgeon"):
                 status.append(s.text)
        return status
    except:
        print("Failed to run the command on that user, Skipping...") 
        
tweet2['statuses'] = tweet2.apply(checkStatus, axis = 1)

tweet2.to_csv("twitter_first_result_status_xx.csv",index=False, encoding = 'utf-8', columns = ['screen_name', 'statuses'])

In [ ]:
import pandas as pd
tweets3 = pd.read_csv("twitter_first_result_status_xx.csv", encoding = "UTF-8", engine='python')

In [ ]:
tweets3.shape

In [ ]:
tweets3.head()

# Non Medical Posts

In [ ]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
consumer_key = "" # Use your own key. To get a key https://apps.twitter.com/
consumer_secret = ""

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)

api = tweepy.API(auth)

In [ ]:
results = []
for tweet in tweepy.Cursor(api.search, q='"job" OR "Hiring" OR "sports" OR "play" OR "rain" OR "book"').items(400):
    results.append(tweet)

print( len(results))

In [ ]:
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    #data_set["retweeted_status"] = [tweet.retweet_status for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_name"] = [tweet.author.name for tweet in results]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in results]
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set
data_set = process_results(results)


In [ ]:
data_set.to_csv("nonmedical.csv", encoding='utf8')

In [ ]:
non_medical_tweets = pd.read_csv("nonmedical.csv", encoding = "UTF-8", engine='python')

In [ ]:
non_medical_tweets.head()

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json


consumer_key = ''
consumer_secret = ''
access_token = ''

access_token_secret = ''



auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)




def process_or_store(tweet):
    print(json.dumps(tweet))
    
    
api = tweepy.API(auth)

#user = api.get_user('EarlofNarwhals')

#for friend in tweepy.Cursor(user.friends).items():
#    process_or_store(friend._json)


### extract the 5 recent statuses of each user
def checkStatus(x):
    try:
        status = []
        dictioanry = {}
        stuff = api.user_timeline(x['screen_name'], count = 5, include_rts = False)
        for s in stuff:
            #if s.str.contains("medicine|meds|medical|infected|health|pill|pills|drug|drugs|surgeon|surgeons|disease|diseases|virus|viruses|bacteria|vaccine|vaccines|doctor|doctors|patient|patients|hospital|clinic|prescription|sick|illness|allergy|fever|flu|autism|cancer|diabetes|HIV|injection|antibiotic|disorder|physician|dentist|gynecologist|surgeon"):
             status.append(s.text)
        #dictioanry['sts'] = status
        #return dictioanry
        return status
    except:
        print("Failed to run the command on that user, Skipping...")
        


non_medical_tweets['statuses'] = non_medical_tweets.apply(checkStatus, axis = 1)

non_medical_tweets.to_csv("twitter_nonexpert_result_status_xx.csv",index=False, encoding = 'utf-8', columns = ['screen_name', 'statuses'])

In [ ]:
non_medical_tweets2 = pd.read_csv("twitter_nonexpert_result_status_xx.csv", encoding = "UTF-8", engine='python')

In [ ]:
print(non_medical_tweets2.shape)

# Read Expert and Non Expert Tweets

In [ ]:
expert = pd.read_csv("projectData/data/twitter_expert.csv", encoding = "UTF-8", engine='python')
non_expert = pd.read_csv("projectData/data/twitter_nonexpert.csv", encoding = "UTF-8", engine='python')

In [ ]:
print(expert.shape)
print(non_expert.shape)

In [ ]:
frame = [expert, non_expert]

In [ ]:
total_tweets =pd.concat(frame)

In [ ]:
print(total_tweets.shape)
total_tweets.head()
#total_tweets.columns[1]

# Shuffle the data

In [ ]:
from sklearn.utils import shuffle
total_tweets = shuffle(total_tweets)


In [ ]:
total_tweets.head()

# Tweet_status Processing

In [ ]:
import re
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
#df = df.dropna(how='any',axis=0)
title_doc = total_tweets[total_tweets.columns[1]].values
corpus =[]
for row in title_doc:
    #remove unwanted data
    title = re.sub ('[^a-zA-Z]', ' ', row)
    title = title.lower()   
    title = title.split()
    #remove stopwords
    stop = stopwords.words('english')
    #word stemming
    title = [lemma.lemmatize(item) for item in title if item not in stop]
    title = ' '.join(title)
    corpus.append(title)
print(corpus[:5])

# Document Term Matrix for tweets

In [ ]:
import numpy as np
vec = CountVectorizer()
X = vec.fit_transform(corpus)
# X = np.array(X)
#data_matrix = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
#print(data_matrix.shape)
y= total_tweets.iloc[:, 2].values
print(X.shape)
y.shape

# split the Dataset into 80% training & 20% testing - Title

In [ ]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# from sklearn import svm
# sv = svm.SVC()
# clf_sv = sv.fit(X_train,y_train)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
clf_rf = rf.fit(X_train,y_train)
print ("Acurracy: ", clf_rf.score(X_test,y_test))
y_pred = clf_rf.predict(X_test)

In [ ]:
print(y_pred )

In [ ]:
y_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(n_estimators=5, max_depth=5, random_state=1)

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
predictions = cross_val_predict(random_forest_clf, X, y)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y, predictions, normalize=True)
plt.show()

In [ ]:
# #############################################################################
# Compute train and test errors
alphas = np.logspace(-5, 1, 60)
enet = linear_model.ElasticNet(l1_ratio=0.7)
train_errors = list()
test_errors = list()
for alpha in alphas:
    enet.set_params(alpha=alpha)
    enet.fit(X_train, y_train)
    train_errors.append(enet.score(X_train, y_train))
    test_errors.append(enet.score(X_test, y_test))

i_alpha_optim = np.argmax(test_errors)
alpha_optim = alphas[i_alpha_optim]
print("Optimal regularization parameter : %s" % alpha_optim)

# Estimate the coef_ on full data with optimal regularization parameter
enet.set_params(alpha=alpha_optim)
coef_ = enet.fit(X, y).coef_

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(2, 1, 1)
plt.semilogx(alphas, train_errors, label='Train error')
plt.semilogx(alphas, test_errors, label='Test error')
plt.vlines(alpha_optim, plt.ylim()[0], np.max(test_errors), color='k',
           linewidth=3, label='Optimum on test')
plt.legend(loc='lower left')
plt.ylim([0, 1.2])
plt.xlabel('Regularization parameter')
plt.ylabel('Performance')



In [ ]:
coef = np.random.randn(500)
# Show estimated coef_ vs true coef
plt.subplot(2, 1, 2)
plt.plot(coef, label='True coef')
plt.plot(coef_, label='Estimated coef')
plt.legend()
plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.26)
plt.show()

In [ ]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt



X1 =X
y1=y


title = "Learning Curves (Random forest)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = RandomForestClassifier(n_estimators=100,n_jobs=-1)
plot_learning_curve(estimator, title, X1, y1, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X1, y1, (0.7, 1.01), cv=cv, n_jobs=4)

plt.show()